In [21]:
import minsearch
import json

In [22]:
with open("documents.json", 'rt') as f_in:
    docs_raw = json.load(f_in)

In [23]:
documents = []

# Flatten the documents
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [24]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [25]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"] # SELECT * WHERE course = 'data-engineering-zoomcamp'; #kind of SQL query for keyword_fields
)

In [26]:
q = "the course has already started, can i still enroll?" #question example

In [27]:
index.fit(documents)

In [28]:
# All these fields como from elastic search and buy default they are all equal to 1.0

boost = {
    "question": 3.0, #the question field is 3 times mr important than the other fields
    'section':0.5 # this filed is 0.5 times less important than the other fields
}

results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'}, #filter for a specific course
    boost_dict=boost,
    num_results=5
)

In [29]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [30]:
from openai import OpenAI
import os

In [31]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [32]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{
        "role": "user", 
        "content": "is it too late to join the course?"
         },
    ]
)

In [34]:
response.choices[0].message.content

"To determine whether it's too late to join a course, you'll need to consider several factors:\n\n1. **Enrollment Deadline**: Check if the course has a specific enrollment deadline. Most institutions and online platforms set a final date by which you must register.\n\n2. **Course Progress**: If the course has already started, find out how far along it is. Some courses may allow late enrollment within a certain window, while others may not.\n\n3. **Institution Policies**: Different educational institutions have different policies about late enrollment. You may need to contact the academic office or the course instructor to see if exceptions can be made.\n\n4. **Course Format**: If the course is self-paced, you might be able to join at any time. In contrast, synchronous courses (where all students move through the material at the same pace) may have stricter deadlines.\n\n5. **Prerequisites**: Make sure you meet any prerequisites for the course. If you join late, you may need to catch up

In [45]:
prompt_template = """
You're a course teaching assistant.Answer the QUESTION based on CONTEXT from the FAQ database.
Use only the facts from CONTEXT when answering when answering the QUESTION.
If the CONTEXT doesn't contain the answer, out NONE

QUESTION: {question}

CONTEXT:{context}

""".strip()

In [42]:
context = ""

for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [43]:
print(context)

section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

section: General course-related questions
question: Course - When will the course start?
answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start wit

In [49]:
prompt = prompt_template.format(question=q, context=context).strip()

In [50]:
print(prompt)

You're a course teaching assistant.Answer the QUESTION based on CONTEXT from the FAQ database.
Use only the facts from CONTEXT when answering when answering the QUESTION.
If the CONTEXT doesn't contain the answer, out NONE

QUESTION: the course has already started, can i still enroll?

CONTEXT:section: General course-related questions
question: Course - Can I still join the course after the start date?
answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

section: General course-related questions
question: Course - Can I follow the course after it finishes?
answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your f

In [51]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{
        "role": "user", 
        "content": prompt
         },
    ]
)

In [53]:
print(response.choices[0].message.content)

Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.
